In [34]:
import glob
import pandas as pd

In [44]:
user_id_map = pd.read_csv('user_github_id_list.tsv', sep='\t', dtype='object').dropna()
user_id_map = user_id_map.drop_duplicates(subset=['user_instance', 'user_id'])

In [45]:
user_id_map.head()

,Unnamed: 0,user_instance,user_id
0,0,rheinze@assemblag.es,109287555889598271
1,1,BrianJAP@mastodon.world,109376150646500709
2,2,joelglasman@fediscience.org,109285335069156233
3,3,brenton_peterson@sciences.social,109269805703778449
4,4,keremd@social.anoxinon.de,108194974464772153


In [11]:
import numpy as np

u,c = np.unique(user_id_map['user_id'].values, return_counts=True)

In [4]:
len(user_id_map)

10595

In [32]:
user_id_map[user_id_map['user_id'] == '109321922930929886']

,Unnamed: 0,user_instance,user_id
4299,4299,Jradin@mas.to,109321922930929886
10101,10101,jradin@mas.to,109321922930929886


In [35]:
files = glob.glob("temp/users_toots_github_*.tsv")

In [36]:
len(files)

63

In [47]:
all_data = []
for file in files:
    data = pd.read_csv(file, sep='\t', dtype='object')
    all_data.append(data)
    
data = pd.concat(all_data)

In [48]:
data.columns

Index(['Unnamed: 0', 'id', 'created_at', 'in_reply_to_account_id', 'uri',
       'replies_count', 'reblogs_count', 'favourites_count', 'content',
       'user_id'],
      dtype='object')

In [49]:
data_with_id = data.merge(user_id_map, on='user_id')

In [51]:
data_with_id = data_with_id.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y'])
data_with_id.head()

,id,created_at,in_reply_to_account_id,uri,replies_count,reblogs_count,favourites_count,content,user_id,user_instance
0,110791111995168546,2023-07-28T09:56:47.934Z,NaN,https://assemblag.es/users/rheinze/statuses/11...,0,0,4,"<p>This looks promising, a French biography of...",109287555889598271,rheinze@assemblag.es
1,110790744739166267,2023-07-28T08:23:24.055Z,NaN,https://assemblag.es/users/rheinze/statuses/11...,0,0,0,NaN,109287555889598271,rheinze@assemblag.es
2,110782162098943816,2023-07-26T20:00:43.356Z,NaN,https://assemblag.es/users/rheinze/statuses/11...,0,0,0,NaN,109287555889598271,rheinze@assemblag.es
3,110780019145721406,2023-07-26T10:55:44.486Z,NaN,https://assemblag.es/users/rheinze/statuses/11...,0,0,0,NaN,109287555889598271,rheinze@assemblag.es
4,110779723869922193,2023-07-26T09:40:38.934Z,NaN,https://assemblag.es/users/rheinze/statuses/11...,0,0,0,NaN,109287555889598271,rheinze@assemblag.es


In [73]:
github_users = pd.read_csv('mastodon_profiles_github.tsv', sep='\t')
github_users.columns = ['index', 'user_instance', 'display_name', 'twitter', 'followers', 'following', 'created_at', 'bio', 'field']
github_users = github_users.drop(columns=['field', 'index'])

fields_users = pd.read_csv('mastodon_profiles_fields.tsv', sep='\t')
fields_users.columns = ['index', 'user_instance', 'field_file']
fields_users = fields_users.drop(columns=['index'])

github_users = github_users.merge(fields_users, on='user_instance', how='left')
github_users = github_users.dropna(subset='field_file')

print(github_users.head())

                 user_instance      display_name twitter  followers following  \
0         rheinze@assemblag.es     Robert Heinze     NaN     1041.0       245   
1         rheinze@assemblag.es     Robert Heinze     NaN     1041.0       245   
2      BrianJAP@mastodon.world  Brian J Peterson     NaN      232.0       239   
3  joelglasman@fediscience.org      Joël Glasman     NaN      864.0      1303   
4  joelglasman@fediscience.org      Joël Glasman     NaN      864.0      1303   

                  created_at  \
0  2022-11-04 00:00:00+00:00   
1  2022-11-04 00:00:00+00:00   
2  2022-11-20 00:00:00+00:00   
3  2022-11-04 00:00:00+00:00   
4  2022-11-04 00:00:00+00:00   

                                                 bio  \
0  <p>Historian, Deutsches Historisches Institut ...   
1  <p>Historian, Deutsches Historisches Institut ...   
2  <p>Historian and professor at Union College in...   
3  <p>historian. global history, humanitarianism,...   
4  <p>historian. global history, humanit

In [76]:
subtopics_map = pd.read_csv('topic2subtopic.csv')
subtopics_map.columns = ['idx', 'level0', 'subtopic']
subtopics_map.head()

,idx,level0,subtopic
0,0,sociology,qualitative research
1,1,sociology,technology policy
2,2,sociology,journalism
3,3,sociology,anthropology
4,4,sociology,criminology


In [60]:
content = open('concepts_unification.txt').read().split('\n\n')
mapconcepts = dict()
for row in content:
    temp = row.split('\n')
    temp2 = []
    for item in temp:
        temp2.append(item.split(" (")[0])

    mapconcepts[temp[0]] = tuple(temp2[1:])

In [74]:
fields_unification = []
for idx, row in github_users.iterrows():
    temp = mapconcepts[row['field_file']]
    fields_unification.append(temp)

In [77]:
fields_unification[:5]

[('african studies',),
 ('history',),
 ('african studies',),
 ('african studies',),
 ('history of science and technology',)]

In [82]:
users_fields = []
for field in fields_unification:
    field = field[0]
    temp = subtopics_map[subtopics_map['subtopic'] == field].iloc[0]['level0']
    users_fields.append(temp)
    #como lidar com mais de uma área?

In [85]:
github_users['field_level0'] = users_fields

In [88]:
mentions = pd.read_csv('github_users_mentions.tsv', sep='\t')

In [89]:
mentions.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'toot_author', 'mentioned_user',
       'toot_id', 'uri', 'content', 'mentioned_user_instance'],
      dtype='object')

In [92]:
nodes = set()
edges = []
for idx, row in mentions.iterrows():
    source = row['toot_author']
    target = row['mentioned_user_instance']
    edges.append((source, target))
    nodes.add(source)
    nodes.add(target)

In [94]:
len(nodes),len(edges)

(46748, 235768)

In [95]:
nodes = list(nodes)
instances = [node.split('@')[1] for node in nodes]

In [109]:
import igraph as ig
g_mentions = ig.Graph(directed=True)

In [110]:
g_mentions.add_vertices(len(nodes))
g_mentions.vs['name'] = nodes
g_mentions.vs['instance'] = instances
g_mentions.add_edges(edges)
g_mentions.simplify()

In [111]:
fields = []
for node in nodes:
    temp = github_users[github_users['user_instance'] == node]
    if len(temp) > 0:
        f = temp.iloc[0]['field_level0']
    else:
        f = 'unknown'
    fields.append(f)

In [112]:
g_mentions.vs['field'] = fields

In [113]:
g_mentions.vs['in_degree'] = g_mentions.degree(mode='IN')

In [114]:
import xnetwork as xnet

In [115]:
xnet.igraph2xnet(g_mentions, 'g_mentions_github_simplify.xnet')

In [116]:
len(set(edges))

106895

In [117]:
g_mentions.ecount()

106642